In [4]:
#seq2seq

id, eid2w, ew2id = 1, {}, {}


with open('enja_datasets/train.en.vocab.4k','r',encoding='utf-8') as f:
    for w in f:
        w = w.strip()
        eid2w[id] = w
        ew2id[w] = id
        id += 1
    
ev = id

id, jid2w, jw2id = 1, {}, {}

with open('enja_datasets/train.ja.vocab.4k','r',encoding='utf-8') as f:
    for w in f:
        w = w.strip()
        jid2w[id] = w
        jw2id[w] = id
        id += 1

jv = id
ev, jv

(4097, 4097)

In [5]:
e_traindata = []
j_traindata = []

with open('enja_datasets/train.en','r',encoding='utf-8') as f:
    for sen in f:
        wl = [ew2id['<s>']]
        for w in sen.strip().split():
            if w in ew2id:
                wl.append(ew2id[w])
            else:
                wl.append(ew2id['<unk>'])
        wl.append(ew2id['</s>'])
        e_traindata.append(wl)


with open('enja_datasets/train.ja','r',encoding='utf-8') as f:
    for sen in f:
        wl = [jw2id['<s>']]
        for w in sen.strip().split():
            if w in jw2id:
                wl.append(jw2id[w])
            else:
                wl.append(jw2id['<unk>'])
        wl.append(jw2id['</s>'])
        j_traindata.append(wl)

len(e_traindata), len(j_traindata)

(50000, 50000)

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [69]:
class MyNMT(nn.Module):
    def __init__(self, jv, ev, k):
        super(MyNMT, self).__init__()
        self.jemb = nn.Embedding(jv, k)
        self.eemb = nn.Embedding(ev, k)
        self.lstm1 = nn.LSTM(k, k, num_layers=2, batch_first=True)
        self.lstm2 = nn.LSTM(k, k, num_layers=2, batch_first=True)
        self.W = nn.Linear(k, ev)
    
    def forward(self, jline, eline):
        x = self.jemb(jline)
        ox, (hnx, cnx) = self.lstm1(x)
        y = self.eemb(eline)
        oy, (hy, cy) = self.lstm2(y, (hnx, cnx))
        out = self.W(oy)
        return out

In [70]:
demb = 200
net = MyNMT(jv, ev, demb).to(device)
optimizer = optim.SGD(net.parameters(),lr=0.01)
criterion = nn.CrossEntropyLoss()


In [71]:
net.train()

for epoch in range(1):
    loss1K = 0.0
    
    for i in range(len(j_traindata)):
        jinput = torch.LongTensor([j_traindata[i][1:]]).to(device)
        einput = torch.LongTensor([e_traindata[i][:-1]]).to(device)
        out = net(jinput, einput)
        gans = torch.LongTensor([e_traindata[i][1:]]).to(device)
        loss = criterion(out[0],gans[0])
        loss1K += loss.item()
        if (i % 100 == 0):
            print(f'{epoch} {i} {loss1K}')
            loss1K = 0.0
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    outfile = "models/nmt-" + str(epoch) + ".model"
    torch.save(net.state_dict(), outfile)
        

0 0 8.29130744934082
0 100 829.5550956726074
0 200 825.1824007034302
0 300 820.3561305999756
0 400 815.1369400024414
0 500 810.3085055351257
0 600 803.0885925292969
0 700 797.4838538169861
0 800 787.4920740127563
0 900 775.9510021209717
0 1000 751.8827557563782
0 1100 725.284451007843
0 1200 697.839653968811
0 1300 666.7477874755859
0 1400 663.0504541397095
0 1500 662.1208090782166
0 1600 652.0442175865173
0 1700 650.3863458633423
0 1800 639.270094871521
0 1900 623.6920185089111
0 2000 623.1360478401184
0 2100 615.6013369560242
0 2200 602.1300315856934
0 2300 604.3598217964172
0 2400 591.9295434951782
0 2500 590.5933175086975
0 2600 579.3468413352966
0 2700 571.353138923645
0 2800 574.5697207450867
0 2900 556.4327208995819
0 3000 554.5551075935364
0 3100 555.3669843673706
0 3200 540.1292889118195
0 3300 545.9324727058411
0 3400 539.772353887558
0 3500 543.2614665031433
0 3600 532.3470277786255
0 3700 531.8613831996918
0 3800 534.8630900382996
0 3900 525.0968906879425
0 4000 524.9192018

In [8]:
# testdata loader

e_testdata = []
with open('enja_datasets/test.en','r',encoding='utf-8') as f:
    for sen in f:
        wl = [ew2id['<s>']]
        for w in sen.strip().split():
            if w in ew2id:
                wl.append(ew2id[w])
            else:
                wl.append(ew2id['<unk>'])
        wl.append(ew2id['</s>'])
        e_testdata.append(wl)
        
j_testdata = []
with open('enja_datasets/test.ja','r',encoding='utf-8') as f:
    for sen in f:
        wl = [jw2id['<s>']]
        for w in sen.strip().split():
            if w in jw2id:
                wl.append(jw2id[w])
            else:
                wl.append(jw2id['<unk>'])
        wl.append(jw2id['</s>'])
        j_testdata.append(wl)

In [75]:


demb = 200
net = MyNMT(jv, ev, demb).to(device)

net.load_state_dict(torch.load('enja_datasets/nmt-19.model'))

esid = ew2id['<s>']
eeid = ew2id['</s>']

net.eval()

with torch.no_grad():
    for i in range(len(j_testdata)):
        jinput = torch.LongTensor([j_testdata[i][1:]]).to(device)
        x = net.jemb(jinput)
        ox, (hnx, cnx) = net.lstm1(x)
        wid = esid
        sl = 0
        while True:
            wids = torch.LongTensor([[wid]]).to(device)
            y = net.eemb(wids)
            oy, (hnx, cnx) = net.lstm2(y, (hnx, cnx))
            oy1 = net.W(oy)
            wid = torch.argmax(F.softmax(oy1[0],dim = 1)).item()
            if (wid == eeid):
                break
            print(eid2w[wid],end=' ')
            sl += 1
            if (sl == 30):
                break
        print()

it was true that they were right at it time . 
he was not poor for music . 
he is kind to her sister . 
we must have off ten minutes ago . 
i will make a mistake to do . 
she lives next door to me . 
i am anxious about what i had been up . 
i am a <unk> of friends . 
this machine is ! 
say that one , you must say it , too . 
she is a teacher . 
i 'm doing it . 
two two years old . 
he made his son the meeting in the examination . 
take care of me . 
the work on the day for our vacation . 
i was not seen from her . 
some people are on the park for the next family . 
i must watch two books . 
that is the movie yesterday is mine . 
it is ten years since i am born . 
i met him on the tokyo left . 
i often go to the river by river . 
he asked his help . 
he fell out from the secret in the door . 
i don 't agree to his proposal . 
we read the letter and more slowly . 
i 'm very glad to see you . 
please help yourself to your father . 
they <unk> the proposal . 
i feel better , too . 
i have 

In [76]:
from nltk.translate.bleu_score import corpus_bleu

myans = [['<s>', 'I', 'am', 'a', 'student', '</s>']]
gold = [['<s>', 'I', 'am', 'a', 'student', '</s>']]

score = corpus_bleu(myans, gold)
score

C:\Users\keisu\AppData\Roaming\Python\Python39\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\keisu\AppData\Roaming\Python\Python39\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\keisu\AppData\Roaming\Python\Python39\site-packages\nltk\translate\bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using 

1.171778691554733e-231

In [9]:
# attention nmt

class MyAttNMT(nn.Module):
    def __init__(self, jv, ev, k):
        super(MyAttNMT, self).__init__()
        self.jemb = nn.Embedding(jv, k)
        self.eemb = nn.Embedding(ev, k)
        self.lstm1 = nn.LSTM(k, k, num_layers=2, batch_first=True)
        self.lstm2 = nn.LSTM(k, k, num_layers=2, batch_first=True)
        self.Wc = nn.Linear(2*k, k)
        self.W = nn.Linear(k, ev)
    
    def forward(self, jline, eline):
        x = self.jemb(jline)
        ox, (hnx, cnx) = self.lstm1(x)
        y = self.eemb(eline)
        oy, (hny, cny) = self.lstm2(y, (hnx, cnx))
        oxl = ox.permute(0,2,1)
        sim = torch.bmm(oy,oxl)
        bs, yws, xws = sim.shape
        sim2 = sim.reshape(bs*yws,xws)
        alpha = F.softmax(sim2,dim=1).reshape(bs,yws,xws)
        ct = torch.bmm(alpha,ox)
        oy1 = torch.cat([ct,oy],dim=2)
        oy2 = self.Wc(oy1)
        
        return self.W(oy2)

In [79]:
demb = 200

net = MyAttNMT(jv, ev, demb).to(device)
optimizer = optim.Adam(net.parameters(),lr=0.01)
criterion = nn.CrossEntropyLoss()

net.train()

for epoch in range(1):
    loss1K = 0.0
    
    for i in range(len(j_traindata)):
        jinput = torch.LongTensor([j_traindata[i][1:]]).to(device)
        einput = torch.LongTensor([e_traindata[i][:-1]]).to(device)
        out = net(jinput, einput)
        gans = torch.LongTensor([e_traindata[i][1:]]).to(device)
        loss = criterion(out[0],gans[0])
        loss1K += loss.item()
        if (i % 100 == 0):
            print(f'{epoch} {i} {loss1K}')
            loss1K = 0.0
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    outfile = "models/nmt-att-" + str(epoch) + ".model"
    torch.save(net.state_dict(), outfile)
        

0 0 8.298240661621094
0 100 615.1582500934601
0 200 520.5831758975983
0 300 507.0234434604645
0 400 496.95163106918335
0 500 494.8906307220459
0 600 487.3170716762543
0 700 492.8523426055908
0 800 472.1480224132538
0 900 479.5804090499878
0 1000 472.59447050094604
0 1100 478.105997800827
0 1200 472.2029023170471
0 1300 463.108350276947
0 1400 465.65382385253906
0 1500 465.85118103027344
0 1600 475.4659752845764
0 1700 463.7798430919647
0 1800 461.6131868362427
0 1900 454.25495433807373
0 2000 460.75872898101807
0 2100 458.7743124961853
0 2200 449.87214374542236
0 2300 457.8096787929535
0 2400 453.6090655326843
0 2500 461.5407814979553
0 2600 463.87257862091064
0 2700 460.51949751377106
0 2800 466.37762117385864
0 2900 443.3595747947693
0 3000 459.37645721435547
0 3100 461.9961106777191
0 3200 444.27423763275146
0 3300 458.38203024864197
0 3400 443.09332716464996
0 3500 468.35825848579407
0 3600 456.18369245529175
0 3700 455.60914397239685
0 3800 458.564088344574
0 3900 459.126272439956

In [ ]:
demb = 200
net = MyAttNMT(jv, ev, demb).to(device)
net.load_state_dict(torch.load('models/nmt-att-0.model'))

esid = ew2id['<s>']
eeid = ew2id['</s>']

net.eval()


with torch.no_grad():
    for i in range(len(j_testdata)):
        jinput = torch.LongTensor([j_testdata[i][1:]]).to(device)
        x = net.jemb(jinput)
        ox, (hnx, cnx) = net.lstm1(x)
        wid = esid
        sl = 0
        while True:
            wids = torch.LongTensor([[wid]]).to(device)
            y = net.eemb(wids)
            oy, (hnx, cnx) = net.lstm2(y,(hnx, cnx))
            ox1 = ox.permute(0,2,1)
            sim = torch.bmm(oy,ox1)
            bs, yws, xws = sim.shape
            sim2 = sim.reshape(bs*yws,xws)
            alpha = F.softmax(sim2,dim=1).reshape(bs, yws, xws)
            ct = torch.bmm(alpha,ox)
            oy1 = torch.cat([ct,oy],dim=2)
            oy2 = net.Wc(oy1)
            oy3 = net.W(oy2)
            wid = torch.argmax(oy3[0]).item()
            if (wid == eeid):
                break
            print(eid2w[wid]," ",end='')
            sl += 1
            if (sl == 30):
                break
        print()

he  is  to  see  to  return  .  
he  is  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  
he  is  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  
i  am  a  <unk>  .  
i  succeed  succeed  is  succeed  .  
i  am  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  
i  am  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  
i  am  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  to  see  
i  won  't  win  .  
i  don  't  say  to  say  to  say  to  say  to  say  to  say  to  say  to  say  to  say  to  say  to  say  to  say  to  say  to  say  
i  dress  the  dress  .  
it  is  it  it  it  it  it  it  it  it  it  it  .  
i 

In [10]:
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence

class MyDataset(Dataset):
    def __init__(self, xdata, ydata):
        self.data = xdata
        self.label = ydata
    def __len__(self):
        return len(self.label)
    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.label[idx]
        return (x,y)

def my_collate_fn(batch):
    images, targets= list(zip(*batch))
    xs = list(images)
    ys = list(targets)
    return xs, ys


In [11]:
batch_size = 100
dataset = MyDataset(j_traindata,e_traindata)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=my_collate_fn)

demb = 200
net = MyAttNMT(jv, ev, demb).to(device)
optimizer = optim.Adam(net.parameters(),lr=0.01)
criterion = nn.CrossEntropyLoss(ignore_index=-1)

max_epoch = 10

net.train()

for epoch in range(max_epoch):
    i = 0
    for xs, ys in dataloader:
        xs1, ys1, ys2 = [], [], []
        
        for k in range(len(xs)):
            tid = xs[k]
            xs1.append(torch.LongTensor(tid[1:]))
            tid = ys[k]
            ys1.append(torch.LongTensor(tid[:-1]))
            ys2.append(torch.LongTensor(tid[1:]))
        
        jinput = pad_sequence(xs1, batch_first=True).to(device)
        einput = pad_sequence(ys1, batch_first=True).to(device)
        gans   = pad_sequence(ys2, batch_first=True,padding_value=-1.0).to(device)
        out = net(jinput, einput)
        loss = criterion(out[0],gans[0])
        for h in range(1, len(gans)):
            loss += criterion(out[h],gans[h])
        print(f'|epochs{epoch}/{max_epoch} | {i} | loss:{loss.item()} |')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
    outfile = "models/nmt-att2-" + str(epoch) + ".model"
    torch.save(net.state_dict(), outfile)
        

|epochs0/10 | 0 | loss:832.0750732421875 |
|epochs0/10 | 1 | loss:794.0260620117188 |
|epochs0/10 | 2 | loss:618.7998657226562 |
|epochs0/10 | 3 | loss:770.0001831054688 |
|epochs0/10 | 4 | loss:611.23046875 |
|epochs0/10 | 5 | loss:625.4752807617188 |
|epochs0/10 | 6 | loss:543.1723022460938 |
|epochs0/10 | 7 | loss:524.4873657226562 |
|epochs0/10 | 8 | loss:527.4093627929688 |
|epochs0/10 | 9 | loss:517.699951171875 |
|epochs0/10 | 10 | loss:505.5751953125 |
|epochs0/10 | 11 | loss:500.6339111328125 |
|epochs0/10 | 12 | loss:496.34527587890625 |
|epochs0/10 | 13 | loss:496.38604736328125 |
|epochs0/10 | 14 | loss:496.4184875488281 |
|epochs0/10 | 15 | loss:496.2947692871094 |
|epochs0/10 | 16 | loss:481.4613037109375 |
|epochs0/10 | 17 | loss:486.93621826171875 |
|epochs0/10 | 18 | loss:478.51873779296875 |
|epochs0/10 | 19 | loss:484.6336669921875 |
|epochs0/10 | 20 | loss:485.5776672363281 |
|epochs0/10 | 21 | loss:487.3521728515625 |
|epochs0/10 | 22 | loss:478.1802978515625 |
|ep

KeyboardInterrupt: 

In [ ]:
import time

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np

###################

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

##################

# Data setting

class MyDataset(Dataset):
    def __init__(self, xdata, ydata):
        self.data = xdata
        self.label = ydata
    def __len__(self):
        return len(self.label)
    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.label[idx]
        return (x,y)

def my_collate_fn(batch):
    images, targets= list(zip(*batch))
    xs = list(images)
    ys = list(targets)
    return xs, ys

###########################

id, eid2w, ew2id = 1, {}, {}
with open('enja_datasets/train.en.vocab.4k','r',encoding='utf-8') as f:
    for w in f:
        w = w.strip()
        eid2w[id] = w
        ew2id[w] = id
        id += 1
ev = id

edata = []
with open('enja_datasets/train.en','r',encoding='utf-8') as f:
    for sen in f:
        wl = [ew2id['<s>']]
        for w in sen.strip().split():
            if w in ew2id:
                wl.append(ew2id[w])
            else:
                wl.append(ew2id['<unk>'])
        wl.append(ew2id['</s>'])
        edata.append(wl)

id, jid2w, jw2id = 1, {}, {}
with open('enja_datasets/train.ja.vocab.4k','r',encoding='utf-8') as f:
    id = 1
    for w in f:
        w = w.strip()
        jid2w[id] = w
        jw2id[w] = id
        id += 1
jv = id

jdata = []
with open('enja_datasets/train.ja','r',encoding='utf-8') as f:
    for sen in f:
        wl = [jw2id['<s>']]
        for w in sen.strip().split():
            if w in jw2id:
                wl.append(jw2id[w])
            else:
                wl.append(jw2id['<unk>'])
        wl.append(jw2id['</s>'])
        jdata.append(wl)

batch_size = 100
dataset = MyDataset(jdata,edata)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=my_collate_fn)

# Define model

class MyAttNMT(nn.Module):
    def __init__(self, jv, ev, k):
        super(MyAttNMT, self).__init__()
        self.jemb = nn.Embedding(jv, k, padding_idx=0)
        self.eemb = nn.Embedding(ev, k, padding_idx=0)
        self.lstm1 = nn.LSTM(k, k, num_layers=2, batch_first=True)
        self.lstm2 = nn.LSTM(k, k, num_layers=2, batch_first=True)
        self.Wc = nn.Linear(2*k, k)
        self.W = nn.Linear(k, ev)
    def forward(self, jline, eline):
        x = self.jemb(jline)
        ox, (hnx, cnx) = self.lstm1(x)
        y = self.eemb(eline)
        oy, (hny, cny) = self.lstm2(y,(hnx, cnx))
        ox1 = ox.permute(0,2,1)
        sim = torch.bmm(oy,ox1)
        bs, yws, xws = sim.shape
        sim2 = sim.reshape(bs*yws,xws)
        alpha = F.softmax(sim2,dim=1).reshape(bs, yws, xws)
        ct = torch.bmm(alpha,ox)
        oy1 = torch.cat([ct,oy],dim=2)
        oy2 = self.Wc(oy1)
        return self.W(oy2)

# model generate, optimizer and criterion setting

demb = 200
net = MyAttNMT(jv, ev, demb).to(device)
optimizer = optim.SGD(net.parameters(),lr=0.01)
criterion = nn.CrossEntropyLoss(ignore_index=-1)

max_epoch = 10

# Learn

net.train()

start_time = time.time()

for ep in range(max_epoch):
    epoch_start_time = time.time()
    i = 0
    for xs, ys in dataloader:
        xs1, ys1, ys2 = [], [], []
        for k in range(len(xs)):
            tid = xs[k]
            xs1.append(torch.LongTensor(tid[1:]))
            tid = ys[k]
            ys1.append(torch.LongTensor(tid[:-1]))
            ys2.append(torch.LongTensor(tid[1:]))
        jinput = pad_sequence(xs1, batch_first=True).to(device)
        einput = pad_sequence(ys1, batch_first=True).to(device)
        gans = pad_sequence(ys2, batch_first=True, padding_value=-1.0).to(device)
        out = net(jinput, einput)
        loss = criterion(out[0],gans[0])
        for h in range(1,len(gans)):
            loss += criterion(out[h],gans[h])
        print(f'|epochs: {ep+1} / {max_epoch} | {i} | loss: {loss.item()} |')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
        
    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time
    total_elapsed_time = epoch_end_time - start_time
    
    print(f"=========Epoch {epoch+1}/{max_epoch} finished in {epoch_duration:.2f} seconds.============")
    outfile = "attnmt2-" + str(ep) + ".model"
    torch.save(net.state_dict(),outfile)


|epochs0/10 | 0 | loss:833.1766967773438 |
|epochs0/10 | 1 | loss:827.957763671875 |
|epochs0/10 | 2 | loss:822.3908081054688 |
|epochs0/10 | 3 | loss:816.6859741210938 |
|epochs0/10 | 4 | loss:809.2805786132812 |
|epochs0/10 | 5 | loss:800.6580810546875 |
|epochs0/10 | 6 | loss:786.65087890625 |
|epochs0/10 | 7 | loss:769.0723266601562 |
|epochs0/10 | 8 | loss:737.5472412109375 |
|epochs0/10 | 9 | loss:694.6349487304688 |
|epochs0/10 | 10 | loss:691.6818237304688 |
|epochs0/10 | 11 | loss:753.5726318359375 |
|epochs0/10 | 12 | loss:714.1585693359375 |
|epochs0/10 | 13 | loss:661.8818969726562 |
|epochs0/10 | 14 | loss:649.713134765625 |
|epochs0/10 | 15 | loss:647.58642578125 |
|epochs0/10 | 16 | loss:648.0253295898438 |
|epochs0/10 | 17 | loss:723.3574829101562 |
|epochs0/10 | 18 | loss:674.1502685546875 |
|epochs0/10 | 19 | loss:614.6467895507812 |
|epochs0/10 | 20 | loss:604.3267211914062 |
|epochs0/10 | 21 | loss:647.8485107421875 |
|epochs0/10 | 22 | loss:603.30908203125 |
|epoch

In [ ]:
net.load_state_dict(torch.load('model/attnmt2-9.model'))

# Learn

esid = ew2id['<s>']
eeid = ew2id['</s>']
net.eval()
with torch.no_grad():
    for i in range(len(j_testdata)):
        jinput = torch.LongTensor([ j_testdata[i][1:] ]).to(device)
        x = net.jemb(jinput)
        ox, (hnx, cnx) = net.lstm1(x)
        wid = esid
        sl = 0
        while True:
            wids = torch.LongTensor([[ wid ]]).to(device)
            y = net.eemb(wids)
            oy, (hnx, cnx) = net.lstm2(y,(hnx, cnx))
            ox1 = ox.permute(0,2,1)
            sim = torch.bmm(oy,ox1)
            bs, yws, xws = sim.shape
            sim2 = sim.reshape(bs*yws,xws)
            alpha = F.softmax(sim2,dim=1).reshape(bs, yws, xws)
            ct = torch.bmm(alpha,ox)
            oy1 = torch.cat([ct,oy],dim=2)
            oy2 = net.Wc(oy1)
            oy3 = net.W(oy2)
            wid = torch.argmax(oy3[0]).item()
            if (wid == eeid):
                break
            print(eid2w[wid]," ",end='')
            sl += 1
            if (sl == 30):
                break
        print()

they  found  it  true  .  
he  was  not  good  at  swimming  in  his  work  .  
he  is  kind  enough  to  cook  for  his  sister  .  
we  must  get  back  before  ten  o  'clock  .  
i  don  't  like  success  to  succeed  success  .  
she  is  going  to  meet  us  next  next  .  
i  am  going  to  tell  you  the  way  to  you  .  
i  am  looking  like  to  play  <unk>  .  
this  game  is  very  good  this  game  .  
tell  me  what  you  did  such  a  thing  .  
she  likes  the  teacher  likes  the  teacher  .  
it  is  a  good  job  ,  that  is  a  job  .  
these  shoes  will  be  able  to  stand  by  two  years  .  
he  made  his  part  present  at  the  meeting  .  
you  must  not  go  out  for  a  drink  .  
the  day  when  it  finished  a  day  ,  i  'm  <unk>  .  
i  was  not  afraid  to  look  her  face  .  
some  people  go  on  on  meals  on  sunday  .  
i  must  watch  two  two  .  
that  is  the  boy  that  i  met  yesterday  .  
ten  years  is  a  long  time  to  wait  .  


In [17]:
import time

In [ ]:
demb = 200
net = MyAttNMT(jv, ev, demb).to(device)
optimizer = optim.Adam(net.parameters(),lr=0.01)
criterion = nn.CrossEntropyLoss(ignore_index=-1)

net.load_state_dict(torch.load('models/attnmt2-9.model'))

pre_trained_epoch = 10

max_epoch = 20

num_epoch = max_epoch - pre_trained_epoch

# Learn

net.train()

start_time = time.time()

for ep in range(num_epoch):
    epoch_start_time = time.time()
    i = 0
    for xs, ys in dataloader:
        xs1, ys1, ys2 = [], [], []
        for k in range(len(xs)):
            tid = xs[k]
            xs1.append(torch.LongTensor(tid[1:]))
            tid = ys[k]
            ys1.append(torch.LongTensor(tid[:-1]))
            ys2.append(torch.LongTensor(tid[1:]))
        jinput = pad_sequence(xs1, batch_first=True).to(device)
        einput = pad_sequence(ys1, batch_first=True).to(device)
        gans = pad_sequence(ys2, batch_first=True, padding_value=-1.0).to(device)
        out = net(jinput, einput)
        loss = criterion(out[0],gans[0])
        for h in range(1,len(gans)):
            loss += criterion(out[h],gans[h])
        print(f'|epochs: {ep+1+pre_trained_epoch} / {max_epoch} | {i} | loss: {loss.item()} |')
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        i += 1
        
    epoch_end_time = time.time()
    epoch_duration = epoch_end_time - epoch_start_time
    total_elapsed_time = epoch_end_time - start_time
    
    print(f"=========Epoch {epoch+pre_trained_epoch}/{max_epoch} finished in {epoch_duration:.2f} seconds.============")
    outfile = "attnmt2-" + str(ep+pre_trained_epoch) + ".model"
    torch.save(net.state_dict(),outfile)

|epochs: 11 / 20 | 0 | loss: 137.22567749023438 |
|epochs: 11 / 20 | 1 | loss: 346.2825012207031 |
|epochs: 11 / 20 | 2 | loss: 357.7021179199219 |
|epochs: 11 / 20 | 3 | loss: 309.77850341796875 |
|epochs: 11 / 20 | 4 | loss: 301.6553039550781 |
|epochs: 11 / 20 | 5 | loss: 299.0032043457031 |
|epochs: 11 / 20 | 6 | loss: 301.0260009765625 |
|epochs: 11 / 20 | 7 | loss: 308.2873840332031 |
|epochs: 11 / 20 | 8 | loss: 297.62646484375 |
|epochs: 11 / 20 | 9 | loss: 301.17974853515625 |
|epochs: 11 / 20 | 10 | loss: 291.4958801269531 |
|epochs: 11 / 20 | 11 | loss: 295.4617614746094 |
|epochs: 11 / 20 | 12 | loss: 294.61749267578125 |
|epochs: 11 / 20 | 13 | loss: 292.10595703125 |
|epochs: 11 / 20 | 14 | loss: 290.0230407714844 |
|epochs: 11 / 20 | 15 | loss: 299.4242248535156 |
|epochs: 11 / 20 | 16 | loss: 299.5684814453125 |
|epochs: 11 / 20 | 17 | loss: 273.213134765625 |
|epochs: 11 / 20 | 18 | loss: 288.35516357421875 |
|epochs: 11 / 20 | 19 | loss: 296.126220703125 |
|epochs: 11

In [25]:
net.load_state_dict(torch.load('models/attnmt2-19.model'))

# Learn

esid = ew2id['<s>']
eeid = ew2id['</s>']
net.eval()
with torch.no_grad():
    for i in range(len(j_testdata)):
        jinput = torch.LongTensor([ j_testdata[i][1:] ]).to(device)
        x = net.jemb(jinput)
        ox, (hnx, cnx) = net.lstm1(x)
        wid = esid
        sl = 0
        while True:
            wids = torch.LongTensor([[ wid ]]).to(device)
            y = net.eemb(wids)
            oy, (hnx, cnx) = net.lstm2(y,(hnx, cnx))
            ox1 = ox.permute(0,2,1)
            sim = torch.bmm(oy,ox1)
            bs, yws, xws = sim.shape
            sim2 = sim.reshape(bs*yws,xws)
            alpha = F.softmax(sim2,dim=1).reshape(bs, yws, xws)
            ct = torch.bmm(alpha,ox)
            oy1 = torch.cat([ct,oy],dim=2)
            oy2 = net.Wc(oy1)
            oy3 = net.W(oy2)
            wid = torch.argmax(oy3[0]).item()
            if (wid == eeid):
                break
            print(eid2w[wid]," ",end='')
            sl += 1
            if (sl == 30):
                break
        print()

they  acknowledged  it  was  last  night  acknowledged  to  .  
he  weren  't  good  good  at  swimming  .  
he  is  as  kind  a  singer  as  as  as  well  walk  .  
you  must  be  back  before  ten  o  'clock  .  
surely  success  success  doubt  desires  success  success  !  
she  is  a  short  house  next  .  
i  'm  about  to  ask  you  to  answer  .  
i  'm  looking  for  a  <unk>  but  <unk>  .  
this  game  is  this  good  use  of  this  game  .  
what  you  did  such  an  eager  thing  .  
she  does  fond  of  the  teacher  like  that  she  like  the  teacher  .  
that  is  the  work  that  that  is  business  job  .  it  's  job  .  it  's  that  job  that  it  is  job  with  work  .  it  's  work  .  it  
these  boots  these  these  these  these  these  boots  these  these  these  boots  these  two  years  will  these  days  .  
he  saved  his  son  to  send  himself  alone  .  
you  have  not  clear  the  eager  without  going  .  
i  have  a  <unk>  <unk>  <unk>  in  the  d